In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct  4 20:48:33 2021

@author: navee
"""

#importing required libraries
import pandas as pd #to create dataframe and export as excel
import selenium
from selenium import webdriver #to access webpages
import time# to provide wait time
from IPython.display import clear_output#to clear progress output
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait #to make driver wait
from selenium.webdriver.support import expected_conditions as EC #to check for condition of web element(clickable,presence,etc)
from selenium.webdriver.chrome.options import Options#to add options to webdrver
from webdriver_manager.chrome import ChromeDriverManager# to download chromedriver to avoid issues
from selenium.webdriver.common.action_chains import ActionChains# to execute actions like drag and drop
from selenium.webdriver.common.keys import Keys#to send zip code input
#adding options to chrome to avoid errors and work in headless mode
options1 = webdriver.ChromeOptions()
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.517 Safari/537.36'
options1.add_argument('user-agent={0}'.format(user_agent))
options1.add_argument("--headless")
options1.add_argument("--start-maximized")
options1.add_argument("--window-size=1920,1080")
# options1.add_argument('--start-fullscreen')
#to access zip code input box in webpage
zip_code_xpath='//div[@class="usurp-zip-input-wrapper"]/div/div/input'
#to access links of each car listed in all pages
cars_link_xpath='//div[@class="visible-vehicle-info"]/h2/a'
#search radius accessing path to change radius
search_radius_xpath='//div[@class="range"]/div[2][@aria-labelledby="multi-lbl-search-radius"]/input'
#to identify number of pages to be scraped
number_of_pages_xpath='//div[@class="d-flex flex-row justify-content-around align-items-center flex-nowrap"]/div/a[last()]'
#search radius to be sent as input using dictionary key value pair
radius={'1':-100,'2':-75,'3':-50,'4':0,'5':25,'6':50,'7':75,'8':100}
#adjusting zip code and search radius
#code block that gets url for listed cars across the pages receives base url as argument
def start(url):
    filename=str(input('Please enter filename without extension'))+'.xlsx'
    if url!='https://www.edmunds.com/cars-for-sale-by-owner/':   #checks if page url has changed(url must look like thishttps://www.edmunds.com/inventory/srp.html?inventorytype=used%2Ccpo&radius=10)
        print(f'Current URL: {url}')#prints current url
        print(f'Number of pages: {x}')# prints number of pages to be scraped
        url_list,missed_links=find_car_links(url,x)#calls function that returns list of pages accessed and missed pages
        print(url_list)#prints accessed urls
        print(missed_links)#prints missed urls
        car_info,cars_missed=car_data(url_list)#calls the function that returns list of car info one by one and missed links/cars
        print(f'Missed cars:{cars_missed}')#prints missed links
        car_info_final=[]
        car_info_final.clear()
        car_info_final=car_info#total info of car containing Name,Price,VIN,Vehicle Summary,Top Features and Specs
        df=pd.DataFrame(car_info_final,columns=['Name','Price','VIN','Vehicle Summary','Top Features & Specs'])#creates dataframe
        print(df)#prints dataframe
        df.to_excel(filename,index=None)#exports data frame
        print('done')#prints done
    else:
        print('No change in URL noticed,restart')#asks user to retry
#code block that returns list of car URLs listed across pages and pages that were missed
def find_car_links(url_ba,x):#receives 
    
    loopend=[]
    b='MissMissMissMissMissMissMissMissMissMissMissMissMissMissMissMissMissMissMissMiss'
    c=[]
    loopend.clear()
    w=1#variable used to give a wait to avoid connection termination after reading data of every 20 pages
    all_links=[]#list to capture all the links of cars across pages
    all_links.clear()#clears list everytime the code runs
    missed_pages=[]#list to capture missed pages
    missed_pages.clear()#clears list everytime the code runs
    for i in range(1,x+1):#loop to loop through pages
        try:
            driver.get(url_ba+'&pagenumber='+str(i))#loads pages one by one
            driver.implicitly_wait(5)#waits till page loads
            links=driver.find_elements_by_xpath(cars_link_xpath)#finds all links in that page adhereing to the listed cars
            link=[e.get_attribute('href') for e in links]#gets the href
            for j in link:
                all_links.append(j)#appends the found links
            print(f'Acquired links of cars Loop number:{w}')#mentions the number of time loop has executed
            if w%20==0:
                print('Waiting to avoid connection termination....Getting links of cars')
                time.sleep(10)#to provide waitiong time
             

        except:
            print('missed')     
            missed_pages.append(url_ba+'&pagenumber='+str(i))#capturing missed URLs
            loopend.append('Miss')
            c=''.join(loopend)
            if b in c:#ends and exports excel if 20 continuous attempts fail
                c=None
                
                break
        w=w+1
    driver.quit()
    return all_links,missed_pages#returning accessed links and missed links
#code block to capture info of each car
def car_data(url_list):#takes list of URLs accessed from previous block        
    driver1=webdriver.Chrome(ChromeDriverManager().install(),options=options1) 
    w1=1##variable used to give a wait to avoid connection termination after reading data of every 20 cars 
    d_list=[]#list to capture car info as a list
    dat_list=[]#to append d_list of each car
    dat_list.clear()
    b='MissMissMissMissMissMissMissMissMissMissMissMissMissMissMissMissMissMissMissMiss'#in case twenty pages continuously are unreachable
    loopend=[]
    c=[]
    loopend.clear()
    missed_car_data=[]#to capture missed URLs
    missed_car_data.clear()
    df=pd.DataFrame(columns=['Name','Price','VIN','Vehicle Summary','Top Features & Specs'])
    for l in url_list:
        try:
            driver1.implicitly_wait(3)
            driver1.get(l)#loads the page and gets data such as(Name,Price,VIN)
            driver1.implicitly_wait(3)
            price=driver1.find_element_by_xpath('//div[@class="col-6"]/div/span').text
            name=driver1.find_element_by_xpath('//div[@class="vdp-group mb-2"]/section/h1').text+driver1.find_element_by_xpath('//div[@class="vdp-group mb-2"]/section/div[2]/span').text
            vin=driver1.find_element_by_xpath('//div[@class="vdp-group mb-2"]/section/div[2]/div/span').text
            vin=vin.replace('VIN: ','')#Removes VIN: part in text
            s=[]#to capture left side of vehicle summary
            s1=[]#to capture right side of vehicle summary
            st=[]#to capture entire summary
            st.clear()
            s1.clear()
            s.clear()
            #replacements to be made in summary as it gets text of child nodes
            replacements=['Based on model year EPA mileage ratings. Use for comparison purposes only. Your mileage will vary depending on how you drive and maintain your vehicle, driving conditions, battery-pack age/condition and other factors.','Based on default model values and available option information. Contact dealer to confirm.']
            #gets count of items in left side of vehicle summary to avoid loop errors
            sleft=len(driver1.find_elements_by_xpath('//div[@class="col-12 col-md-12"]/section[@class="vehicle-summary mb-0_5 text-gray-darker"]/div/div[1]/div/div[2]'))
            #gets count of items in right side of vehicle summary to avoid loop errors
            sright=len(driver1.find_elements_by_xpath('//div[@class="col-12 col-md-12"]/section[@class="vehicle-summary mb-0_5 text-gray-darker"]/div/div[2]/div/div[2]'))
            
            #loop to get summary text from left side
            for j in range(1,sleft+1):

                    text=driver1.find_element_by_xpath(f'//div[@class="col-12 col-md-12"]/section[@class="vehicle-summary mb-0_5 text-gray-darker"]/div/div[1]/div[{j}]/div[2]').text
                    text=text.replace(replacements[0],'')
                    text=text.replace(replacements[1],'')
                    text=text.replace('\n','')
                    s.append(text)
            #loop to get summary text from right side

            for j in range(1,sright+1):

                    text1=driver1.find_element_by_xpath(f'//div[@class="col-12 col-md-12"]/section[@class="vehicle-summary mb-0_5 text-gray-darker"]/div/div[2]/div[{j}]/div[2]').text
                    text1=text.replace(replacements[0],'')
                    text1=text.replace(replacements[1],'')
                    text1=text.replace('\n','')
                    s1.append(text1)
            #creating total summary for the car
            st.extend(s)
            st.extend(s1)
            st=','.join(st)
            #to get count of number of top features and specs available to avoid loop errors
            n_features=driver1.find_elements_by_xpath('//div[@class="pl-1 pl-md-0 mb-0_5 row"]/div')
            feature_list=[]#list to capture the features
            feature_list.clear()
            #loop to acquire features
            for i in range(1,len(n_features)+1):
                t1=driver1.find_element_by_xpath(f'//div[@class="pl-1 pl-md-0 mb-0_5 row"]/div[{i}]/div').text
                t2=driver1.find_element_by_xpath(f'//div[@class="pl-1 pl-md-0 mb-0_5 row"]/div[{i}]/div[@class="features-with-collapse"]').text
                t2=t2.replace('\n',' , ')
                feature_list.append(t1+':'+t2)#creates Feature heading:feature contents
            feature_list=','.join(feature_list)
            d_list=[[name,price,vin,st,feature_list]]#creating lsit of car info
            dat_list.extend(d_list)#extending it to array which gives entire info at the end
            print(f'Acquired data of car {w1}')
            loopend.append('Captured')
            
            if w1%20==0:
                print('Waiting to avoid connection termination....Getting data of cars')
                time.sleep(10)#providing wait time
            

        except:
            print('missed')
            missed_car_data.append(l)#capturing missed data
            loopend.append('Miss')
            c=''.join(loopend)
            if b in c:#ends and exports excel if 20 continuous attempts fail
                c=None
                driver1.quit()
                break
        w1=w1+1
    driver1.quit()#quittinf webdriver
    return dat_list,missed_car_data#returning car info and missed list
#input for zip code
zip_code=input('Please enter your zip code:')
#input for radius
radius_inp=str(input('Choose a radius option(miles, 1-6):\n1:10 miles\n2:25 miles\n3:50 miles\n4:75 miles\n5:100 miles\n6:200 miles\n7:500 miles\n8:Nation Wide\n'))
#creating webdriver in below mentioned way to avoid errors because of version
driver=webdriver.Chrome(ChromeDriverManager().install(),options=options1) 
#loading main page
driver.get('https://www.edmunds.com/cars-for-sale-by-owner/')
#waiting for page to load entirely
driver.implicitly_wait(10)
#accessing zip code input box
zip_inp=driver.find_element_by_xpath('//div[@class="d-flex"]/input')
#deleting already present zip code and inputting preferred one
zip_inp.send_keys(Keys.BACKSPACE)
zip_inp.send_keys(Keys.BACKSPACE)
zip_inp.send_keys(Keys.BACKSPACE)
zip_inp.send_keys(Keys.BACKSPACE)
zip_inp.send_keys(Keys.BACKSPACE)
zip_inp.send_keys(zip_code)
#accessing search radius slider
slider = driver.find_element_by_xpath(search_radius_xpath)
#creating ActionChains object
action = ActionChains(driver)
#moving the slider to preferred radius
action.drag_and_drop_by_offset(slider,radius[radius_inp], 0).perform()
driver.implicitly_wait(7)
#acquiring the number of pages
number_of_pages= WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, number_of_pages_xpath)))
#converting number of pages to integer
x=int(number_of_pages.text)
driver.refresh()
driver.implicitly_wait(7)                                                                                                                    
#acquiring the url of current page
url=driver.current_url
print(url)
#calling the start funcion with base url as argument

a=start(url)



Please enter your zip code:85001
Choose a radius option(miles, 1-6):
1:10 miles
2:25 miles
3:50 miles
4:75 miles
5:100 miles
6:200 miles
7:500 miles
8:Nation Wide
1




====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\navee\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


https://www.edmunds.com/inventory/srp.html?inventorytype=used%2Ccpo&radius=10
Please enter filename without extensionnb
Current URL: https://www.edmunds.com/inventory/srp.html?inventorytype=used%2Ccpo&radius=10
Number of pages: 476
Acquired links of cars Loop number:1




====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\navee\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


['https://www.edmunds.com/gmc/sierra-1500/2018/vin/3GTU2NEC0JG419619/?radius=10', 'https://www.edmunds.com/gmc/sierra-1500/2021/vin/3GTU9FEL7MG111336/?radius=10', 'https://www.edmunds.com/lincoln/continental/2020/vin/1LN6L9RP5L5600652/?radius=10', 'https://www.edmunds.com/chevrolet/silverado-1500/2021/vin/3GCUYHEL5MG187068/?radius=10', 'https://www.edmunds.com/ford/f-150/2018/vin/1FTEW1CP2JFE60701/?radius=10', 'https://www.edmunds.com/alfa-romeo/stelvio/2018/vin/ZASFAKNNXJ7B83561/?radius=10', 'https://www.edmunds.com/ford/mustang/2019/vin/1FA6P8TH1K5181269/?radius=10', 'https://www.edmunds.com/kia/soul/2019/vin/KNDJP3A51K7661123/?radius=10', 'https://www.edmunds.com/ford/f-150/2018/vin/1FTEW1CPXJKE44088/?radius=10', 'https://www.edmunds.com/ford/f-150/2019/vin/1FTEW1CP9KKC96050/?radius=10', 'https://www.edmunds.com/ford/f-150/2018/vin/1FTEW1CGXJKF57863/?radius=10', 'https://www.edmunds.com/ford/f-150/2019/vin/1FTEW1C55KKE95305/?radius=10', 'https://www.edmunds.com/jeep/gladiator/2020/v